In [35]:
import torch
import torchaudio
from transformers import WhisperModel, WhisperFeatureExtractor, AutoFeatureExtractor

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Whisper Features Extraction

In [63]:
model_path="openai/whisper-medium"
whisper_model = WhisperModel.from_pretrained(model_path)
whisper_model = whisper_model.encoder
whisper_model = whisper_model.to(device)
whisper_model.eval()

WhisperEncoder(
  (conv1): Conv1d(80, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(1024, 1024, kernel_size=(3,), stride=(2,), padding=(1,))
  (embed_positions): Embedding(1500, 1024)
  (layers): ModuleList(
    (0-23): 24 x WhisperEncoderLayer(
      (self_attn): WhisperAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (activation_fn): GELUActivation()
      (fc1): Linear(in_features=1024, out_features=4096, bias=True)
      (fc2): Linear(in_features=4096, out_features=1024, bias=True)
      (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    )
  )
  (layer_norm): LayerNorm((1024,), eps=

In [64]:
feature_extractor = AutoFeatureExtractor.from_pretrained(model_path)

In [71]:
#filepath = "dataset_music_background_classifier/music/0b1inHMAr6o-00001.wav"
filepath = "dataset_music_background_classifier/clean/0b1inHMAr6o-00001.wav"
audio_data, sr = torchaudio.load(filepath)

In [73]:
# Extract Embedding
inputs = feature_extractor(
    audio_data.squeeze(), sampling_rate=16000, return_tensors="pt"
)
input_features = inputs.input_features
input_features = input_features.to(device)
with torch.no_grad():
    file_embedding = whisper_model(input_features).last_hidden_state
file_embedding = file_embedding.to(device)

 # Classification

In [77]:
from model.whisper_dense import WhisperDenseModel
classifier_model = WhisperDenseModel()

In [78]:
# Download checkpoint from gs://vai-ml/models/music_background_classifier/whisper-medium.tar.bz
# and extract to checkpoints folder
resume_path = "checkpoints/whisper-medium/training-30-05-12h36m26s/model_best.pth"
checkpoint = torch.load(resume_path)
classifier_model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [79]:
output = classifier_model(file_embedding).squeeze()
output = output.reshape(-1)
prediction = output > 0.5
prediction

tensor([False])